In [2]:
import torch
a=[13.5,12.5,11.5]
a=torch.tensor(a)
a.round()



tensor([14., 12., 12.])

In [2]:
torch.tensor([-2.5000 , -1.5000,   -0.5000,    0.5000 ,   1.5000  ,  2.5000]).round()

tensor([-2., -2., -0.,  0.,  2.,  2.])

In [3]:
X=torch.tensor([[[1,2,3,4],[1,2,3,4]],[[1,2,3,4],[1,2,3,4]]])
Y=torch.tensor([1])

X*Y

tensor([[[1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[1, 2, 3, 4],
         [1, 2, 3, 4]]])

In [10]:
import torch
x =torch.tensor([[[1,2,3,4]]])
x.shape
x.expand(1,4,4)
for i in x[0,0,:]:
    print(i)

tensor(1)
tensor(2)
tensor(3)
tensor(4)


In [1]:
out_scale=torch.tensor(0.234)
in_scale=torch.rand(1,1,384)
channel_nums=384
SCALE=torch.rand(1,1,384)
x_q=torch.rand(16,197,384)
x_q_sum=x_q.sum(dim=-1)
std_x_q =  torch.sqrt(#std标准差也就是加入alpha后每一行的标准差，
                channel_nums * (x_q**2).sum(dim=-1) - x_q.sum(dim=-1)**2)
print(x_q_sum.shape)
for i in range(x_q.shape[2]):#x_q:[1,197,384]
    x_q[:,:,i]=(x_q[:,:,i]*channel_nums-x_q_sum)/std_x_q
    
for i in range(x_q.shape[1]):
    x_q[:,i,:]=x_q[:,i,:]*in_scale

beta=torch.rand(384)



X_q=torch.rand(1,197,384)
print(X_q.shape)
X_Mean=X_q.sum(dim=-1).unsqueeze(-1)*torch.ones(384)
print(X_Mean.shape,X_Mean)


Bias=torch.rand(384)
Bias=Bias.reshape(1,-1,1)
print(Bias.shape)
Bias=Bias*torch.ones(197)
print(Bias.transpose(-1,-2).shape,Bias)

NameError: name 'torch' is not defined

In [3]:
import torch
batch=16
x_q=torch.rand(batch,197,384)#构造输入
in_scale=torch.rand(1,1,384)
out_scale=torch.rand(1)
channel_nums=x_q.shape[-1]#获取通道数
print(x_q[0,:,:])

x_q=(x_q/in_scale).round()
M1=x_q.sum(dim=-1)
print(M1.shape)
print(M1[:,1].shape)
print(M1[:,1].reshape(16,-1,1).shape)
#然后让每一行的点减去对应行的sum
std_x_q = torch.sqrt(#std标准差也就是加入alpha后每一行的标准差，
                channel_nums * (x_q**2).sum(dim=-1) - x_q.sum(dim=-1)**2)#[1,197]
for i in range(M1.shape[-1]):#对于每一行来说，都是[16,384]维，对于M1的每一行来说，都是[16]维
    x_q[:,i,:]=(x_q[:,i,:]-M1[:,i].reshape(batch,-1))/std_x_q[:,i].reshape(batch,-1)


Gama=torch.rand(1,384)
Beta=torch.rand(1,384)
for i in range(x_q.shape[1]):#对于每一行来说
    x_q[:,i,:]=(x_q[:,i,:]*Gama+Beta).round()
# x=torch.tensor([[[1,2,3],[4,5,6]]])
# M1=x.sum(dim=-1)
# print(x)
# print(M1)
# print(x[:,1,:]-M1[:,1])
a=torch.pow(2,torch.tensor(32))
topk=(1,)
print(topk)


print(Gama.dtype)

tensor([[0.8284, 0.8615, 0.9001,  ..., 0.4303, 0.8895, 0.7683],
        [0.1498, 0.9806, 0.2665,  ..., 0.1643, 0.7077, 0.8185],
        [0.8216, 0.4132, 0.8983,  ..., 0.3648, 0.8573, 0.3017],
        ...,
        [0.5421, 0.3475, 0.2392,  ..., 0.4069, 0.5456, 0.9214],
        [0.9650, 0.2893, 0.0048,  ..., 0.8635, 0.5091, 0.3144],
        [0.7948, 0.3334, 0.4086,  ..., 0.1863, 0.5976, 0.7749]])
torch.Size([16, 197])
torch.Size([16])
torch.Size([16, 1, 1])
(1,)
torch.float32


In [6]:
import torch
#研究acuracy是如何计算的
output=torch.rand(16,1000)#16个batch，1000个类的概率
topk=(1,5)
maxk=max(topk)
_, pred = output.topk(maxk, 1, True, True)

In [48]:
import torch
out_scale=torch.tensor(0.0234)
Gama=((torch.rand(1,1,384)/out_scale*torch.pow(2, torch.tensor(32))).round()/torch.pow(2, torch.tensor(32))).round()
Gama
Beta=((torch.rand(1,1,384)/out_scale*torch.pow(2, torch.tensor(32))).round()/torch.pow(2, torch.tensor(32))).round()
with open ('Scale_Bias.txt','a') as ff:
    for i in range(Gama.shape[-1]):
        ff.write('%02x%02x'%(int(Gama[0,0,i].item())&0xff,int(Beta[0,0,i].item())&0xff))
        ff.write("\n")
ff.close()



In [27]:
a=torch.tensor(100.1).item()
int(a)
a.type()

AttributeError: 'float' object has no attribute 'type'

In [49]:
num=10
b=-12
print('%02x%02x'%(num,b))

0a-c


In [55]:
for x in range(-4,5):
    print ("dec:%d hex:%02X" % (x, x & 0xff))

dec:-4 hex:FC
dec:-3 hex:FD
dec:-2 hex:FE
dec:-1 hex:FF
dec:0 hex:00
dec:1 hex:01
dec:2 hex:02
dec:3 hex:03
dec:4 hex:04


In [27]:
import torch
x_q=(torch.linspace(-197*384,0,197*384)).reshape(1,197,384)
print(x_q)
with open ('Xq_LayerNorm.txt','a') as ff:
    for i in range(x_q.shape[-2]):
        for j in range(x_q.shape[-1]):
            ff.write('%02x'%(int(x_q[0,i,j].item())&0xff))
            ff.write("\n")
ff.close()

tensor([[[-7.5648e+04, -7.5647e+04, -7.5646e+04,  ..., -7.5267e+04,
          -7.5266e+04, -7.5265e+04],
         [-7.5264e+04, -7.5263e+04, -7.5262e+04,  ..., -7.4883e+04,
          -7.4882e+04, -7.4881e+04],
         [-7.4880e+04, -7.4879e+04, -7.4878e+04,  ..., -7.4499e+04,
          -7.4498e+04, -7.4497e+04],
         ...,
         [-1.1510e+03, -1.1500e+03, -1.1490e+03,  ..., -7.7001e+02,
          -7.6901e+02, -7.6801e+02],
         [-7.6701e+02, -7.6601e+02, -7.6501e+02,  ..., -3.8601e+02,
          -3.8501e+02, -3.8401e+02],
         [-3.8301e+02, -3.8201e+02, -3.8101e+02,  ..., -2.0000e+00,
          -1.0000e+00,  0.0000e+00]]])


In [6]:
import torch
fuck=torch.tensor([[[1,2,3],[4,5,6]]])
fuck_sum=fuck.sum(dim=-1)
print(fuck_sum)

fuck=torch.tensor([  -2.,    0.,    2.,  -76.,  -46.,  -62.,   20.,  -64.,    2.,  -96.,
           0.,    2.,    0.,  -62., -108.,   46.,  -32.,   12.,   48.,    2.,
         -14.,   14.,    0.,   -2.,  -20.,  -20.,  -12.,  146.,   34.,   -2.,
           0.,    0.,    4.,   46.,    0.,  -92.,    0.,   -2.,   12.,   28.,
          -2.,   14.,    0.,  124.,   16.,   16.,  -30.,   30.,   12.,   50.,
          12.,   -4.,   80.,   48.,    4.,    4.,    2.,   32.,   32.,   30.,
         -20.,    0.,  -12.,   32.,    4.,  -50.,    4.,  -14.,  -12.,    2.,
          52.,  -16.,  -76.,  -80.,  -36.,   76.,   96.,    0.,  -12.,    0.,
          66.,    2.,   16.,    0.,  -66.,   14.,    0.,   32.,   14.,    2.,
           4.,   12.,    0.,   16.,    0.,   -4.,   22.,   12.,   16.,   20.,
           4.,    0.,   64.,  -48.,  -64.,   12.,   -2.,   14.,  -14.,    0.,
          36.,  -48.,  -14.,   14.,  -12.,    0.,   12.,    0.,   92.,   20.,
         -50., -126.,    2.,   48.,  -12.,  -24.,  -32.,   22.,  -12.,   34.,
        -126.,   14.,  -12.,    0.,  -76.,    0.,   66.,    2.,  -20.,    2.,
          -2.,   12.,   82.,  -16.,   -2.,   12.,    4.,    4.,   32.,  -30.,
          32.,   16.,    0.,    2.,   -2.,  -14.,  -12.,   16.,  -20.,  124.,
          -2.,    2.,   66.,   20.,  -12.,    0.,   62.,   80.,   44.,  -46.,
          44.,   -4.,  -14.,  -30.,    2.,   32.,   20.,  -16.,   -2.,    0.,
         -66.,   -4.,  -64.,  -12.,   20.,    0.,    4.,  -12.,  -82.,  -48.,
         -14.,  -14.,   -4., -156.,  -48.,  -10.,   66.,  -76.,    4.,  -14.,
          62.,   12.,    0.,  -64.,   -2.,  -12.,   -2.,    4.,  -34.,   -2.,
          16.,   -2.,   14.,    2.,    0.,  -16.,   12.,   14.,    0.,   50.,
          -4.,   16.,   32.,   -2.,  -14.,  -12.,  -92.,  -48.,    2.,  -52.,
          -2.,   -4.,   80.,   16.,  -28.,  -30.,   -4.,   32.,   -2.,   -4.,
          16.,  -16.,   -4.,  -32.,   46.,  -16.,    2.,    4.,  -50.,  100.,
          20.,    2.,  -50.,  -98.,   -2.,  -12.,  238.,   12.,   -4.,    2.,
          -2.,   44.,   46.,   30.,   20.,   14.,  -20.,   12.,   -2.,  -62.,
          -4.,    0.,    0.,  -32.,  -30.,   12.,    2.,    0.,   78.,    4.,
           2.,   20.,    0.,    4.,    0.,   -4.,  -30.,    0.,   -2.,  -64.,
          30.,    0.,   14.,   -4.,    4.,  -12.,  -16.,   34.,  -12.,   -2.,
          58.,   46.,    4.,    0.,  -14.,    0.,   12.,   78.,   80.,  -20.,
          34.,  -12.,    0.,   30.,    2.,   -2.,    0.,    0.,   28.,    4.,
          32.,    0.,   -2.,   78.,    6.,   48.,   -4.,  -14.,    0.,   34.,
         -34., -784.,  -12.,   -2.,   16.,   68.,    2.,   66.,    4.,  -30.,
          12.,    2.,   62.,   12.,   30.,   28.,  -14.,    4.,   -4.,  -14.,
          96.,    4.,   32.,  -62.,   32.,  -80.,  -14.,  -64.,    2.,    2.,
           0.,   16.,    4.,  -12.,    0.,  -66.,   24.,  -20.,  -14.,  -24.,
           2.,    0.,   20.,    0.,   36.,   12.,    4.,   -4.,    2.,   -4.,
           0.,    2.,   -4.,   16.], device='cuda:0')
fuck.sum()

tensor([[ 6, 15]])


tensor(30., device='cuda:0')